## Importing Required Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from glob import glob
from PIL import Image
import os
import random
import cv2

import os
import random
import keras

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, AveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from keras.metrics import PrecisionAtRecall,Recall 
#Model Analysis
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tensorflow.keras.metrics import TruePositives, FalsePositives, TrueNegatives, FalseNegatives, Precision, Recall, AUC

## Extracting Files

In [3]:
# c = '../input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID/images'
# n = '../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/images'
# p = '../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Viral Pneumonia/images'

c = '/content/drive/Othercomputers/My MacBook Air/Upgrad ML/LJMU_MS/Model Building/COVID-19_Radiography_Dataset/COVID/images'
n = '/content/drive/Othercomputers/My MacBook Air/Upgrad ML/LJMU_MS/Model Building/COVID-19_Radiography_Dataset/Normal/images'
p = '/content/drive/Othercomputers/My MacBook Air/Upgrad ML/LJMU_MS/Model Building/COVID-19_Radiography_Dataset/Viral Pneumonia/images'

In [4]:
random.seed(42)
filenames = random.sample(os.listdir(c), 2000) + random.sample(os.listdir(n), 2000) + os.listdir(p)

In [5]:
len(filenames)

5345

### Classes
* 0 -> Normal
* 1 -> Viral Pneumonia
* 2 -> COVID

In [6]:
%%time
categories = []
for filename in filenames:
    category = filename.split('-')[0]
    if category == 'COVID':
        categories.append(str(2))
    elif category == 'Viral Pneumonia':
        categories.append(str(1))
    else:
        categories.append(str(0))

CPU times: user 3.64 ms, sys: 52 µs, total: 3.7 ms
Wall time: 3.56 ms


## Converting to Dataframe

In [7]:
%%time
for i in range(len(filenames)):
    if 'COVID' in filenames[i]:
        filenames[i] = os.path.join(c, filenames[i])
    elif 'Viral Pneumonia' in filenames[i]:
        filenames[i] = os.path.join(p, filenames[i])
    else:
        filenames[i] = os.path.join(n, filenames[i])
    
    
df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

CPU times: user 11.6 ms, sys: 866 µs, total: 12.4 ms
Wall time: 12.7 ms


In [8]:
df.head()

,filename,category
0,/content/drive/Othercomputers/My MacBook Air/U...,2
1,/content/drive/Othercomputers/My MacBook Air/U...,2
2,/content/drive/Othercomputers/My MacBook Air/U...,2
3,/content/drive/Othercomputers/My MacBook Air/U...,2
4,/content/drive/Othercomputers/My MacBook Air/U...,2


## Splitting Train, Validation & Test Data

In [9]:
train_data, test_valid_data = train_test_split(df, test_size=0.3, random_state = 42, shuffle=True, stratify=df['category'])
train_data = train_data.reset_index(drop=True)
test_valid_data = test_valid_data.reset_index(drop=True)

In [10]:
test_data, valid_data = train_test_split(test_valid_data, test_size=0.5, random_state = 42,
                                         shuffle=True, stratify=test_valid_data['category'])
test_data = test_data.reset_index(drop=True)
valid_data = valid_data.reset_index(drop=True)

In [11]:
%%time
train_data_gen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

CPU times: user 46 µs, sys: 8 µs, total: 54 µs
Wall time: 56 µs


In [12]:
%%time
train_generator = train_data_gen.flow_from_dataframe(
    train_data, 
    x_col='filename',
    y_col='category',
    target_size=(224,224),
    class_mode='categorical',
    batch_size=15
)

Found 3741 validated image filenames belonging to 3 classes.
CPU times: user 180 ms, sys: 245 ms, total: 425 ms
Wall time: 1.15 s


In [13]:
%%time
valid_data_gen = ImageDataGenerator(rescale=1./255)

valid_generator = valid_data_gen.flow_from_dataframe(
    valid_data, 
    x_col='filename',
    y_col='category',
    target_size=(224,224),
    class_mode='categorical',
    batch_size=15
)

Found 802 validated image filenames belonging to 3 classes.
CPU times: user 41.6 ms, sys: 51.7 ms, total: 93.3 ms
Wall time: 251 ms


# Stacking ensembling of saved models

In [14]:
model1 = tf.keras.models.load_model('/content/drive/Othercomputers/My MacBook Air/Upgrad ML/LJMU_MS/Model Building/Kaggle run up with all metrices/VGG16_30epoc_0.5drop_7_3.h5')

model2 = tf.keras.models.load_model('/content/drive/Othercomputers/My MacBook Air/Upgrad ML/LJMU_MS/Model Building/Kaggle run up with all metrices/ResNet_30epoc_0.5drop_7_3.h5')

model3 = tf.keras.models.load_model('/content/drive/Othercomputers/My MacBook Air/Upgrad ML/LJMU_MS/Model Building/Kaggle run up with all metrices/DenseNet_30_epoc_0.5drop_7_3.h5')

In [15]:
! pip install deepstack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
from deepstack.base import KerasMember

member1 = KerasMember(name="model1", keras_model=model1, train_batches=train_generator, val_batches=valid_generator)
member2 = KerasMember(name="model2", keras_model=model2, train_batches=train_generator, val_batches=valid_generator)
member3 = KerasMember(name="model3", keras_model=model3, train_batches=train_generator, val_batches=valid_generator)

54/54 [==============================] - 5s 91ms/step


In [17]:

from deepstack.ensemble import StackEnsemble
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

#Ensure you have the scikit-learn version >= 0.22 installed
print("sklearn version must be >= 0.22. You have:", sklearn.__version__)

stack = StackEnsemble()

# 2nd Level Meta-Learner
estimators = [
    ('rf', RandomForestClassifier(verbose=0, n_estimators=400, max_depth=50, n_jobs=100, min_samples_split=30)),
    ('etr', ExtraTreesClassifier(verbose=0, n_estimators=400, max_depth=40, n_jobs=100, min_samples_split=20))
]
# 3rd Level Meta-Learner
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression(),cv= 5, n_jobs= -1, passthrough = True)

stack.model = clf
stack.add_members([member1, member2, member3])
stack.fit()
stack.describe(metric=sklearn.metrics.accuracy_score)

sklearn version must be >= 0.22. You have: 1.0.2
Calling predict
model1 - accuracy_score: 0.9726
model2 - accuracy_score: 0.9972
model3 - accuracy_score: 0.9950
StackEnsemble accuracy_score: 0.9863


0.986284289276808

In [19]:

from deepstack.ensemble import DirichletEnsemble

wAvgEnsemble = DirichletEnsemble()
wAvgEnsemble.add_members([member1, member2, member3])
wAvgEnsemble.fit()
wAvgEnsemble.describe()


model1 - Weight: 0.3489 - roc_auc_score: 0.9726
model2 - Weight: 0.2796 - roc_auc_score: 0.9972
model3 - Weight: 0.3715 - roc_auc_score: 0.9950
DirichletEnsemble roc_auc_score: 0.9991
